# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()


,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
data2 = data.groupby(['CustomerID','ProductName'])[['Quantity']].sum()
data2.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:

data3 = pd.pivot_table(data2, index=['ProductName'], columns=['CustomerID'], values='Quantity', aggfunc=np.sum, fill_value=0)
data3.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
distances = pdist(data3.T, 'euclidean')
z = squareform(distances)
print(distances)
print("...............")
print(z)
distances2= pd.DataFrame(1/(1 + squareform(pdist(data3.T, 'euclidean'))), index=data3.columns, columns=data3.columns)
distances2.head()

[ 11.91637529  10.48808848  11.22497216 ... 304.13812651 305.16389039
 303.10889132]
...............
[[  0.          11.91637529  10.48808848 ... 228.62851966 239.
  229.77380181]
 [ 11.91637529   0.          11.74734012 ... 228.01096465 239.03765394
  229.70415756]
 [ 10.48808848  11.74734012   0.         ... 228.08112592 238.26665734
  229.77380181]
 ...
 [228.62851966 228.01096465 228.08112592 ...   0.         304.13812651
  305.16389039]
 [239.         239.03765394 238.26665734 ... 304.13812651   0.
  303.10889132]
 [229.77380181 229.70415756 229.77380181 ... 305.16389039 303.10889132
    0.        ]]


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
most = distances2[33].sort_values(ascending=False)[1:6]
most

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
data2.head()

records =list(most.index)
records
data5= data2.loc[records]
data5.head()

Quantity
CustomerID ProductName                         
264        Apricots - Halves                  1
           Apricots Fresh                     1
           Bacardi Breezer - Tropical         1
           Bagel - Plain                      1
           Banana - Leaves                    1

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
data6 = data5.groupby(['ProductName'])[['Quantity']].sum()
data6 = data6.sort_values(by=['Quantity'], ascending=False)
data6.head()

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
t33=data3[33]
t33=t33[(t33==0)]
product=data6.merge(t33, on='ProductName')[:5]
product


,Quantity,33
ProductName,,
Butter - Unsalted,3,0
Wine - Ej Gallo Sierra Valley,3,0
Soup - Campbells Bean Medley,3,0
Wine - Blue Nun Qualitatswein,3,0
Chicken - Soup Base,2,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [15]:
recomm={}
lis=data.CustomerID.unique()
l=sorted(lis)

for i in l:
    mos = distances2[i].sort_values(ascending=False)[1:6]
    record =list(mos.index)
    data5= data2.loc[record]
    data6 = data5.groupby(['ProductName'])[['Quantity']].sum()
    data6 = data6.sort_values(by=['Quantity'], ascending=False)
    ti=data3[i]
    ti=ti[(ti==0)]
    product=data6.merge(ti, on='ProductName')[:5]
    re={i:product.index[0:5]}
    recomm.update(re)
    
recomm

{33: Index(['Butter - Unsalted', 'Wine - Ej Gallo Sierra Valley',
        'Soup - Campbells Bean Medley', 'Wine - Blue Nun Qualitatswein',
        'Chicken - Soup Base'],
       dtype='object', name='ProductName'),
 200: Index(['Soup - Campbells Bean Medley', 'Muffin - Carrot Individual Wrap',
        'Bay Leaf', 'Pork - Kidney', 'Wanton Wrap'],
       dtype='object', name='ProductName'),
 264: Index(['Soupfoamcont12oz 112con', 'Wine - Two Oceans Cabernet',
        'Bread - Italian Roll With Herbs', 'Veal - Inside, Choice',
        'Fish - Scallops, Cold Smoked'],
       dtype='object', name='ProductName'),
 356: Index(['Veal - Inside, Choice', 'Lamb - Ground', 'Pomello',
        'Butter - Unsalted', 'Tart Shells - Sweet, 4'],
       dtype='object', name='ProductName'),
 412: Index(['Olive - Spread Tapenade', 'Sprouts - Baby Pea Tendrils',
        'Wine - Blue Nun Qualitatswein', 'Pepper - Black, Whole',
        'Soup - Campbells Bean Medley'],
       dtype='object', name='ProductName'

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [16]:
data9=pd.DataFrame(recomm)
data9

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Butter - Unsalted,Soup - Campbells Bean Medley,Soupfoamcont12oz 112con,"Veal - Inside, Choice",Olive - Spread Tapenade,Butter - Unsalted,Olive - Spread Tapenade,Cod - Black Whole Fillet,Bar - Granola Trail Mix Fruit Nut,"Veal - Inside, Choice",...,Cheese - Wine,Muffin Batt - Choc Chk,"Pork - Loin, Bone - In",Fenngreek Seed,"Placemat - Scallop, White","Soup - Campbells, Lentil",Skirt - 29 Foot,"Lamb - Whole, Fresh",Crackers - Trio,Beans - Wax
1,Wine - Ej Gallo Sierra Valley,Muffin - Carrot Individual Wrap,Wine - Two Oceans Cabernet,Lamb - Ground,Sprouts - Baby Pea Tendrils,"Sauce - Gravy, Au Jus, Mix",Sprouts - Baby Pea Tendrils,Wine - Ej Gallo Sierra Valley,Wine - Magnotta - Belpaese,Sardines,...,Muffin - Carrot Individual Wrap,Papayas,Potatoes - Idaho 100 Count,"Wine - Red, Colio Cabernet",Rum - Mount Gay Eclipes,"Cheese - Brie,danish",Beans - Kidney White,Bread - Raisin Walnut Oval,Pernod,Sherry - Dry
2,Soup - Campbells Bean Medley,Bay Leaf,Bread - Italian Roll With Herbs,Pomello,Wine - Blue Nun Qualitatswein,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,Yoghurt Tubes,Appetizer - Mushroom Tart,Wine - Ej Gallo Sierra Valley,...,"Lentils - Red, Dry",Meldea Green Tea Liquor,Pants Custom Dry Clean,Grenadine,Tofu - Firm,Wiberg Super Cure,Milk - 1%,Chips Potato Salt Vinegar 43g,Tea - Jasmin Green,Pork - Hock And Feet Attached
3,Wine - Blue Nun Qualitatswein,Pork - Kidney,"Veal - Inside, Choice",Butter - Unsalted,"Pepper - Black, Whole","Peas - Pigeon, Dry","Veal - Inside, Choice","Pepper - Black, Whole","Peas - Pigeon, Dry","Pepper - Black, Whole",...,Meldea Green Tea Liquor,Wine - Chablis 2003 Champs,Cake - Box Window 10x10x2.5,Ice Cream Bar - Oreo Cone,Sobe - Tropical Energy,Pants Custom Dry Clean,Cheese - Taleggio D.o.p.,Tea - English Breakfast,Pastry - Choclate Baked,Pears - Bosc
4,Chicken - Soup Base,Wanton Wrap,"Fish - Scallops, Cold Smoked","Tart Shells - Sweet, 4",Soup - Campbells Bean Medley,Bar - Granola Trail Mix Fruit Nut,"Pepper - Black, Whole",Soupfoamcont12oz 112con,"Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,...,Coffee - Dark Roast,Mussels - Frozen,Pie Filling - Cherry,Halibut - Fletches,"Cheese - Boursin, Garlic / Herbs",Shrimp - 31/40,Sprouts - Baby Pea Tendrils,Wine - Redchard Merritt,"Peas - Pigeon, Dry","Soup - Campbells, Cream Of"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [17]:
dista = pdist(data3.T, 'correlation')
z = squareform(dista)
print(dista) 
print("...............")
print(z)
dista2= pd.DataFrame(1/(1 + squareform(pdist(data3.T, 'correlation'))), index=data3.columns, columns=data3.columns)
dista2.head()

[1.02521101 0.8879431  0.95940007 ... 0.98268152 1.01783582 0.9766223 ]
...............
[[0.         1.02521101 0.8879431  ... 1.05568047 1.03506774 1.03222313]
 [1.02521101 0.         1.0010674  ... 0.99899044 1.06495391 1.04609733]
 [0.8879431  1.0010674  0.         ... 0.98609845 0.93897585 1.03717867]
 ...
 [1.05568047 0.99899044 0.98609845 ... 0.         0.98268152 1.01783582]
 [1.03506774 1.06495391 0.93897585 ... 0.98268152 0.         0.9766223 ]
 [1.03222313 1.04609733 1.03717867 ... 1.01783582 0.9766223  0.        ]]


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.493776,0.529677,0.510360,0.506951,0.507217,0.487410,0.497932,0.499215,0.502499,...,0.493385,0.484376,0.493486,0.479867,0.499870,0.479764,0.493197,0.486457,0.491384,0.492072
200,0.493776,1.000000,0.499733,0.498575,0.482172,0.481921,0.519139,0.480599,0.499131,0.505459,...,0.504719,0.496170,0.517884,0.508954,0.492581,0.484514,0.534491,0.500253,0.484272,0.488735
264,0.529677,0.499733,1.000000,0.497579,0.513573,0.490305,0.503243,0.480936,0.502028,0.490242,...,0.508803,0.490805,0.499750,0.499162,0.517743,0.525278,0.511845,0.503500,0.515736,0.490875
356,0.510360,0.498575,0.497579,1.000000,0.490143,0.489778,0.501933,0.507803,0.482686,0.486354,...,0.499105,0.486660,0.498662,0.502228,0.494239,0.496086,0.484207,0.502204,0.506401,0.500631
412,0.506951,0.482172,0.513573,0.490143,1.000000,0.497577,0.505111,0.512470,0.493947,0.507777,...,0.488109,0.529502,0.492033,0.495264,0.504752,0.500064,0.481259,0.502418,0.505997,0.515015


In [18]:
rec={}
lis=data.CustomerID.unique()
l=sorted(lis)

for i in l:
    mos = dista2[i].sort_values(ascending=False)[1:6]
    record =list(mos.index)
    data5= data2.loc[record]
    data6 = data5.groupby(['ProductName'])[['Quantity']].sum()
    data6 = data6.sort_values(by=['Quantity'], ascending=False)
    ti=data3[i]
    ti=ti[(ti==0)]
    product=data6.merge(ti, on='ProductName')[:5]
    re={i:product.index[0:5]}
    rec.update(re)
    
rec

{33: Index(['Knife Plastic - White', 'Muffin - Zero Transfat', 'Banana Turning',
        'Crush - Cream Soda', 'Veal - Osso Bucco'],
       dtype='object', name='ProductName'),
 200: Index(['Otomegusa Dashi Konbu', 'Milk Powder', 'Potatoes - Idaho 100 Count',
        'Crackers - Trio', 'Pail With Metal Handle 16l White'],
       dtype='object', name='ProductName'),
 264: Index(['Water - Mineral, Natural', 'Wine - Toasted Head',
        'Snapple - Iced Tea Peach', 'Pickerel - Fillets',
        'Garbag Bags - Black'],
       dtype='object', name='ProductName'),
 356: Index(['Cheese - Taleggio D.o.p.', 'Coconut - Shredded, Sweet',
        'Cheese - Cheddarsliced', 'Ocean Spray - Kiwi Strawberry',
        'Olives - Kalamata'],
       dtype='object', name='ProductName'),
 412: Index(['Cake - Mini Cheesecake', 'Butter - Unsalted',
        'Salmon - Atlantic, Skin On', 'Wine - Hardys Bankside Shiraz',
        'Gloves - Goldtouch Disposable'],
       dtype='object', name='ProductName'),
 464: 

In [19]:
data10=pd.DataFrame(rec)
data10

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Knife Plastic - White,Otomegusa Dashi Konbu,"Water - Mineral, Natural",Cheese - Taleggio D.o.p.,Cake - Mini Cheesecake,Bread - Bistro White,Olive - Spread Tapenade,Dried Figs,"Fish - Scallops, Cold Smoked",Cookies - Assorted,...,Vanilla Beans,Wine - Fume Blanc Fetzer,Spice - Peppercorn Melange,"Beef - Ground, Extra Lean, Fresh",Beef - Striploin Aa,Lettuce - Treviso,"Wine - Red, Colio Cabernet","Water, Tap",Cheese - Taleggio D.o.p.,Juice - Orange
1,Muffin - Zero Transfat,Milk Powder,Wine - Toasted Head,"Coconut - Shredded, Sweet",Butter - Unsalted,Knife Plastic - White,Apricots Fresh,Zucchini - Yellow,Napkin White - Starched,Wine - Blue Nun Qualitatswein,...,Tomatoes Tear Drop,Hersey Shakes,French Pastry - Mini Chocolate,Tuna - Salad Premix,Zucchini - Yellow,"Yogurt - Blueberry, 175 Gr",Veal - Inside,Bananas,Squid U5 - Thailand,Pork - Hock And Feet Attached
2,Banana Turning,Potatoes - Idaho 100 Count,Snapple - Iced Tea Peach,Cheese - Cheddarsliced,"Salmon - Atlantic, Skin On",Wonton Wrappers,Squid U5 - Thailand,"Rum - Coconut, Malibu","Mushroom - Porcini, Dry",Pickerel - Fillets,...,Ice Cream Bar - Hageen Daz To,Potatoes - Idaho 100 Count,Scallop - St. Jaques,Pastry - Raisin Muffin - Mini,Langers - Ruby Red Grapfruit,Bread - Calabrese Baguette,Soupfoamcont12oz 112con,Wine - Redchard Merritt,"Peas - Pigeon, Dry",Longos - Grilled Chicken With
3,Crush - Cream Soda,Crackers - Trio,Pickerel - Fillets,Ocean Spray - Kiwi Strawberry,Wine - Hardys Bankside Shiraz,Broom - Corn,"Pasta - Penne, Rigate, Dry",Sobe - Tropical Energy,Wine - Pinot Noir Latour,Beer - Rickards Red,...,Otomegusa Dashi Konbu,Coffee - Dark Roast,"Beef - Tenderlion, Center Cut",Wine - Ruffino Chianti,Table Cloth 54x72 White,Extract - Lemon,Peas - Frozen,Chocolate - Compound Coating,Salmon Steak - Cohoe 8 Oz,Cattail Hearts
4,Veal - Osso Bucco,Pail With Metal Handle 16l White,Garbag Bags - Black,Olives - Kalamata,Gloves - Goldtouch Disposable,Meldea Green Tea Liquor,"Cheese - Boursin, Garlic / Herbs","Spoon - Soup, Plastic",Pork - Inside,"Hickory Smoke, Liquid",...,Squid U5 - Thailand,Blackberries,Water - Green Tea Refresher,Bananas,"Sauce - Gravy, Au Jus, Mix",Coffee - Irish Cream,Sprouts - Baby Pea Tendrils,Longos - Grilled Salmon With Bbq,Veal - Osso Bucco,Campari


los valores no coinciden al cambar la metrica